# Prototype for simple fits using gv_lsqfit

### This gives a simple implementation gv_lsqfit. It has 2 types of fits implemented.
#### Also, gives outliers and prediction using fit function (with errors).
### How to use this:
- To try out a fit with a new function, 
    - Just define f_func
    - Gather data into structures.
    - Perform the fits.


## Index:
* [functions](# func)
* [fit1](# fit1)
* [fit2](# fit2)

In [1]:

import numpy as np
import scipy as sp
import pandas as pd
import glob
import os
import sys

# Graphics
%matplotlib notebook
import matplotlib.pyplot as plt
# plt.style.use('ggplot')

import itertools


import lsqfit
import gvar as gv

# Code for neat formatting.
from IPython.display import display, HTML, Markdown
def printmd(string):
    display(Markdown(string))


In [2]:
sys.path.append('.')
from fit_modules import *

## Data structures:
- all_dta : storing full data. Useful for plot.
- dta : storing data to fit. Dictionary {'x':,'y':gvar(val(err))}
- p0 : Initial parameter guess.

-------------------------------------------------
-------------------------------------------------
## Simple peak fit of data
<a id = ' fit1'></a>

In [3]:
## Peak fit 
# def f_func(x,p):
#     '''
#     f(x)=chi_p + a1 (x-U_p)^2 + a2 (x-U_p)^3  + a3 (x-U_p)^4
#     '''
#     x=x['x']
#     a1,a2,a3=p['a1'],p['a2'],p['a3']
#     chi_p,Up=p['chi_p'],p['Up']
    
#     ans=chi_p + a1 *(x-Up)**2 + a2 *(x-Up)**3  + a3 *(x-Up)**4
#     return ans



def f_func(x,p):
    '''
    f(x)=chi_p + a1 (x-U_p)^2 + a2 (x-U_p)^3  + a3 (x-U_p)^4
    '''
    x=x['x']
    
    # Code to dynamically define variables    
    for key,val in p.items():
        exec(key + '=val')
    
#     a1,a2,a3=p['a1'],p['a2'],p['a3']
#     chi_p,Up=p['chi_p'],p['Up']
    
    ans=chi_p + a1 *(x-Up)**2 + a2 *(x-Up)**3  + a3 *(x-Up)**4
    return ans

In [4]:
# all_dta

In [7]:
# df3.loc[0:10]
# df3

In [17]:
# Extract data from file.
data_dir='test_data/'
df2=pd.read_csv(data_dir+'SUS_1',sep='\t',names=['U','L','sus','dsus']).reset_index().drop('index',1)

# Trim data
df3=df2[df2.L==32][['U','sus','dsus']].reset_index().drop('index',1)
df=df3.copy()
all_dta={'x':df.U.values,'y':gv.gvar(df.sus.values,df.dsus.values)}

df=df3[(df3.U>0.35) & (df3.U<0.44)]
xdata,ydata,yerr=df.U.values,df.sus.values,df.dsus.values
x,y={'x':xdata},{'y':gv.gvar(ydata,yerr)}

# ### Actual fitting of data.
# # Initial guesses for parameters.
par={'chi_p':1,'Up':0.4,'a1':1,'a2':1,'a3':0.04}

# fit=f_perform_fit(all_dta,x,y,par,f_func,verbose=True,plot=True,full_data=False,error_band=False,semilog=False)




In [25]:
# print df3.U.values
min_gap=len(par)


size=df3.U.values.shape[0]
range_lst=[(i,j) for i in range(size) for j in range(i+min_gap,size)]
# print range_lst

fits_list=[]
for i in range_lst:
    df=df3.loc[i[0]:i[1]]
#     print df
    xdata,ydata,yerr=df.U.values,df.sus.values,df.dsus.values
    x,y={'x':xdata},{'y':gv.gvar(ydata,yerr)}

    ### Actual fitting of data.
    # Initial guesses for parameters.
    par={'chi_p':1,'Up':0.4,'a1':1,'a2':1,'a3':0.04}

    fit=f_perform_fit(all_dta,x,y,par,f_func,verbose=False,concise=True,plot=False,full_data=False,error_band=False,semilog=False)

    if fit.chi2/fit.dof<3.0:
        print i
        fits_list.append(fit)
#     print fit
    

(0, 8)
(1, 6)
(1, 7)
(1, 8)
(2, 7)
(2, 8)
(2, 13)
(2, 14)
(3, 8)
(3, 13)
(4, 11)
(4, 12)
(4, 13)
(5, 10)
(5, 11)
(5, 12)
(5, 13)
(5, 14)
(5, 15)
(6, 12)
(6, 13)
(6, 14)
(6, 15)
(7, 13)
(7, 14)
(7, 15)
(8, 14)
(8, 15)
(8, 16)
(9, 14)
(9, 15)
(9, 16)
(10, 15)
(10, 16)
(11, 16)


In [ ]:
# print f_outlier_list(fit)
# df.iloc[f_outlier_list(fit),:]


In [27]:
fit.p

ExtendedDict([('a1', -259(65)),
              ('a3', -13343(8604)),
              ('a2', 3128(1458)),
              ('Up', 0.4038(76)),
              ('chi_p', 3.353(38))])

To do : 
- do the same for covar fit
- Take single range data, amplify errors, pick value within range and try out fits.